# Import everything


In [1]:
from typing import Tuple
import torch
import numpy as np
from torch import nn
from tqdm import tqdm
from torch.utils.data import DataLoader
from settings import cfg
from helper import boilerplate, debug, loader


Initialized


# Get the dataset

In [2]:
dataset_image, dataset_label = loader.load_data(cfg.train_paths, mmap_mode='c')
print(f'''Stats:
    | Number of punching: {dataset_label.sum()}
    | Number of not-punching: {dataset_label.shape[0] - dataset_label.sum()}''')


Stats:
    | Number of punching: 20332
    | Number of not-punching: 20332


# Boilerplate Code

In [3]:
Data = boilerplate.TrainingDataset


# Model

In [4]:
from settings.cfg import device, model, transforms, criterion, optimizer, idx_gen, batch_size

debug.print_model_size(model)


Param size: 491.257MB
Buffer size: 0.021MB


# Train

In [5]:
def train(train_idx: np.ndarray) -> 'Tuple(float, float)':
    model.train()
    train = Data(dataset_image, dataset_label, train_idx)
    train_dataloader = DataLoader(train, batch_size=batch_size)
    total_loss_train = 0
    total_accumulate_train = 0
    for image, label in tqdm(train_dataloader):
        image = image.to(device, dtype=torch.float)
        label = label.to(device, dtype=torch.uint8)

        image = transforms(image)
        output = model(image)

        batch_loss = criterion(output, label)
        total_loss_train += batch_loss.item()

        accumulate = (output.argmax(dim=1) == label).sum()
        total_accumulate_train += accumulate.item()

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        model.zero_grad()

    return total_loss_train, total_accumulate_train


In [6]:
def judge(judge_idx: np.ndarray) -> 'Tuple(float, float)':
    model.eval()
    judge = Data(dataset_image, dataset_label, judge_idx)
    judge_dataloader = DataLoader(judge, batch_size=batch_size)

    total_loss_judge = 0
    total_accumulate_judge = 0
    with torch.no_grad():
        for image, label in tqdm(judge_dataloader):
            image = image.to(device, dtype=torch.float)
            label = label.to(device, dtype=torch.uint8)

            output = model(image)
            batch_loss = criterion(output, label)
            total_loss_judge += batch_loss.item()

            accumulate = (output.argmax(dim=1) == label).sum()
            total_accumulate_judge += accumulate.item()

    return total_loss_judge, total_accumulate_judge

In [7]:
min_judge_loss = float('inf')


for epoch, (train_idx, judge_idx) in enumerate(idx_gen.split(dataset_label)):
    print(f'''Starting epoch {epoch+1}
        | Train size: {train_idx.shape[0]}
        | Judge size: {judge_idx.shape[0]}''')
    total_loss_train, total_accumulate_train = train(train_idx)
    total_loss_judge, total_accumulate_judge = judge(judge_idx)

    print(
        f'''Epoch: {epoch+1} 
        | Train Loss: {total_loss_train / len(train_idx):.3f}
        | Train Accuracy: {total_accumulate_train/len(train_idx):.3f}
        | Val Loss: {total_loss_judge/len(judge_idx):.3f}
        | Val Accuracy: {total_accumulate_judge/len(judge_idx):.3f}'''
    )
    if min_judge_loss > total_loss_judge/len(judge_idx):
        min_judge_loss = total_loss_judge/len(judge_idx)
        torch.save(model.state_dict(), cfg.model_path)
        print(f"Save model because val loss improve loss {min_judge_loss:.3f}")


Starting epoch 1
        | Train size: 36597
        | Judge size: 4067


100%|██████████| 64/64 [00:06<00:00,  9.90it/s]


Epoch: 1 
        | Train Loss: 0.005
        | Train Accuracy: 0.867
        | Val Loss: 0.979
        | Val Accuracy: 0.501


RuntimeError: Parent directory /home/phuonghd/NHAT/BallPunchAIModel/result does not exist.